## Quarterbacks

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from fantasy_football import Fantasy

# Import fantasy class which will be used for building datasets
football = Fantasy()

First, we pull 2022 offense data and clean it

In [19]:
df, y = Fantasy.read_data('2021')

Offense_2020
Offense_Proj
Offense_Prior_Actuals
Offense_Proj


In [20]:
# print(y)

           Player  points
0        J. Allen  417.58
1      J. Herbert  395.76
2      P. Mahomes  374.66
3        T. Brady  386.74
4       J. Burrow  328.24
..            ...     ...
370  E. St. Brown   11.20
371     P. Taylor   15.20
372      K. Smith    8.70
373    G. Gilbert    7.76
374      D. Brown   16.10

[375 rows x 2 columns]


In [12]:
# df_year_2, y_2 = Fantasy.read_data('2022')

Split data by positional group

In [4]:
QB_data = football.makeRegularData(df, "QB")


Next, we pull more advanced data for rushing and passing

In [40]:
# Passing advanced
passing_advanced = football.getAdvancedStats("passing", "2021")

Now we are ready to merge the datasets to incorporate the more advance statistics

In [109]:
QB1 = football.addAdvanced(QB_data, passing_advanced)
QB_merged = football.simpleMerge(QB1, y)
QB_final = football.fix_columns_QB(QB_merged)
QB_final = QB_final.loc[:,~QB_final.columns.duplicated()]
QB_final = QB_final[QB_final['GP'] != '-']


In [112]:
type(QB_final)

pandas.core.frame.DataFrame

In [114]:
QB_model_X = QB_final.loc[:, ~QB_final.columns.isin(['Player', 'Name', 'Pos', 'Tm', 'points'])]
QB_model_Y = QB_final.loc[:, QB_final.columns.isin(['points'])]

In [116]:
QB_model_X.columns

Index(['GP', 'Comp', 'Inc', 'Passing_Yds', 'Passing_Td', 'Int', 'Pic6', 'Sks',
       'Passing_1st', 'Att', 'Rushing_Yds', 'Rushing_Td', 'Rushing_1st',
       'Total', 'Lost', 'Rk', 'Age', 'G', 'GS', 'Cmp', 'Yds', 'IAY', 'IAY/PA',
       'CAY', 'CAY/Cmp', 'CAY/PA', 'YAC', 'YAC/Cmp'],
      dtype='object')

In [117]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(QB_model_X, QB_model_Y,
                                   random_state=104, 
                                   test_size=0.2, 
                                   shuffle=True)

In [78]:
# X_train

In [118]:
X_train.columns

Index(['GP', 'Comp', 'Inc', 'Passing_Yds', 'Passing_Td', 'Int', 'Pic6', 'Sks',
       'Passing_1st', 'Att', 'Rushing_Yds', 'Rushing_Td', 'Rushing_1st',
       'Total', 'Lost', 'Rk', 'Age', 'G', 'GS', 'Cmp', 'Yds', 'IAY', 'IAY/PA',
       'CAY', 'CAY/Cmp', 'CAY/PA', 'YAC', 'YAC/Cmp'],
      dtype='object')

In [119]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

ValueError: Unknown label type: 'continuous'

### CONTRACTS AND OTHER DATA (not yet completed)

We are not using the player contracts yet but here is where we scrape that data

In [7]:
table_contracts = pd.read_html('https://www.spotrac.com/nfl/contracts/sort-value/offense/all-time/free-agent-2021/limit-100/')

player_contracts = table_contracts[0]
player_contracts

,Rank,Player,Signed Age,Yrs,Value,AAV,Sign Bonus,G'teed @ Sign,Practical G'teed
0,1,Jay Cutler Quarterback | 2014-2020 (FA: 2021)...,30,7,"$126,700,000","$18,100,000",NaN,"$38,000,000","$54,000,000"
1,2,Cam Newton Quarterback | 2015-2020 (FA: 2021),26,5,"$103,800,000","$20,760,000","$22,500,000","$41,000,000","$60,000,000"
2,3,Andy Dalton Quarterback | 2014-2020 (FA: 2021...,26,6,"$96,000,000","$16,000,000","$12,000,000","$17,000,000","$17,000,000"
3,4,Kirk Cousins Quarterback | 2018-2020 (FA: 2021),29,3,"$84,000,000","$28,000,000","$3,000,000","$84,000,000","$84,000,000"
4,5,Ryan Tannehill Quarterback | 2015-2020 (FA: 2...,26,4,"$77,000,000","$19,250,000","$11,500,000","$21,500,000","$45,000,000"
...,...,...,...,...,...,...,...,...,...
95,96,Paxton Lynch Quarterback | 2016-2020 (FA: 202...,22,4,"$9,476,296","$2,369,074","$5,091,852","$7,884,074","$7,884,074"
96,97,Lee Smith Tight End | 2018-2020 (FA: 2021) O...,30,3,"$9,000,000","$3,000,000",NaN,"$3,000,000","$3,000,000"
97,97,Sammy Watkins Wide Receiver | 2020-2020 (FA: ...,26,1,"$9,000,000","$9,000,000","$1,000,000","$8,000,000","$8,000,000"
98,97,Donte Moncrief Wide Receiver | 2019-2020 (FA:...,25,2,"$9,000,000","$4,500,000","$3,500,000","$3,500,000","$3,500,000"
